# 1.编码器-解码器结构

In [1]:
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch as th
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import datetime
from DL.mnist_dnn import target

In [2]:
#编码器
class EncoderRNN(nn.Module):
    def __init__(self,input_size,hidden_size,dropout_p=0.1):
        super(EncoderRNN,self).__init__()
        self.hidden_size = hidden_size #隐藏层状态的维度
        self.embedding = nn.Embedding(input_size,hidden_size) #嵌入层，用于将输入的索引序列转换为密集的向量表示
        self.rnn= nn.RNN(input_size=hidden_size,hidden_size=hidden_size,num_layers=1,batch_first=True) #RNN循环神经网络层，用于处理序列数据
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self,input):
        x = self.embedding(input) #将输入序列嵌入为密集的向量表示
        x = self.dropout(x) #在嵌入向量上应用dropout进行随机失活
        output, hidden = self.rnn(x) #通过RNN层处理嵌入向量，得到输出和最终隐藏状态
        return output, hidden

In [5]:
encoder = EncoderRNN(input_size=10,hidden_size=5) #假设输入序列的词汇表大小为10，隐藏状态的维度为5
input_seq = th.tensor([[0,1,2,3,4,5,6,7,8,9]]) #生成一个输入序列
output, hidden = encoder(input_seq) #通过编码器进行前向传播
print('输入向量的维度：',input_seq.size())
print('输出向量的维度：',output.size())
print('最终隐藏状态的维度：',hidden.size())

输入向量的维度： torch.Size([1, 10])
输出向量的维度： torch.Size([1, 10, 5])
最终隐藏状态的维度： torch.Size([1, 1, 5])


In [16]:
MAX_LENGTH = 10

PAD_token = 0
SOS_token = 1
EOS_token = 2

In [10]:
#解码器
class DecoderRNN(nn.Module):
    def __init__(self,hidden_size,output_size):
        super(DecoderRNN,self).__init__()
        self.embedding = nn.Embedding(output_size,hidden_size) #嵌入层，用于将目标序列的索引转换为密集的向量表示
        self.rnn = nn.RNN(hidden_size,hidden_size,num_layers=1,batch_first=True) #RNN循环神经网络层，用于处理序列数据
        self.output = nn.Linear(hidden_size,output_size) #全连接层，用于将隐藏状态映射到输出序列的概率分布

    def forward(self,encoder_outpus,encoder_hidden,target_tensor=None):
        batch_size = encoder_outpus.size(0)
        decoder_input = th.empty(batch_size,1,dtype=th.long).fill_(SOS_token) #创建一个初始的解码器输入，填充为起始标记 开始词元，表示开始生成一个序列
        decoder_hidden = encoder_hidden #解码器的初始隐藏状态为编码器的最终隐藏状态
        decoder_outputs = [] #用于存储解码器的输出序列
        for i in range(MAX_LENGTH):
            decoder_output,decoder_hidden = self.forward_step(decoder_input,decoder_hidden) #对解码器进行一步前向传播
            decoder_outputs.append(decoder_output) #将解码器的输出添加到输出序列中
            if target_tensor is not None: #强制教学，使用目标序列的下一个标记作为解码器的输入
                decoder_input = target_tensor[:,i].unsqueeze(1)
            else: #没有目标序列，使用解码器的输出作为下一个输入
                _,topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()
        decoder_outputs = th.cat(decoder_outputs,dim=1) #将解码器的输出序列拼接起来
        decoder_outputs = F.softmax(decoder_outputs,dim=-1) #对解码器的输出应用softmax函数，得到概率分布
        return decoder_outputs,decoder_hidden,None #返回解码器的输出序列、最终隐藏状态和注意力权重
    def forward_step(self,decoder_input,decoder_hidden):
        x = self.embedding(decoder_input) #将解码器输入嵌入为密集的向量表示
        x = F.relu(x) #通过ReLU激活函数对嵌入向量进行非线性变换
        x,hidden = self.rnn(x,decoder_hidden) #通过RNN层处理嵌入向量，得到输出和最终隐藏状态
        output = self.output(x)
        return output,hidden #返回解码器的输出和最终隐藏状态

In [11]:
decoder = DecoderRNN(hidden_size=5,output_size=10) #假设隐藏状态的维度为5，输出序列的词汇表大小为10
target_seq = th.tensor([[0,1,2,3,4,5,6,7,8,9]]) #生成一个目标序列
encoder_outputs, encoder_hidden = encoder(input_seq) #通过编码器进行前向传播
output, hidden,_ = decoder(encoder_outputs,encoder_hidden,input_seq)#通过解码器进行前向传播
print('输出向量的维度：',output.size())

输出向量的维度： torch.Size([1, 10, 10])


In [17]:
#Attention代码实现
class Attention(nn.Module):
    def __init__(self,hidden_size):
        super(Attention,self).__init__()
        self.Wa = nn.Linear(hidden_size,hidden_size) #线性变换矩阵，用于将编码器的输出映射到注意力权重
        self.Ua = nn.Linear(hidden_size,hidden_size) #线性变换矩阵，用于将解码器的隐藏状态映射到注意力权重
        self.Va = nn.Linear(hidden_size,1) #线性变换矩阵，用于将注意力权重映射到标量

    def forward(self,query,keys):
        scores = self.Va(th.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1) #将注意力分数的维度调整为(batch_size,1,seq_len)
        weights = F.softmax(scores,dim=-1)  #对注意力分数应用softmax函数，得到注意力权重
        context = th.bmm(weights,keys) #根据注意力权重加权计算上下文向量
        return context,weights

In [20]:
#带注意力的解码器
class AttentionDecoderRNN(nn.Module):
    def __init__(self,hidden_size,output_size,dropout_p=0.1):
        super(AttentionDecoderRNN,self).__init__()
        self.embedding = nn.Embedding(output_size,hidden_size) #嵌入层，用于将目标序列的索引转换为密集的向量表示
        self.attention = Attention(hidden_size) #注意力机制
        self.rnn = nn.RNN(2*hidden_size,hidden_size,batch_first=True)  #2*hidden_size表示输入特征的维度，hidden_size表示隐藏状态的维度
        self.output = nn.Linear(hidden_size,output_size) #全连接层，用于将隐藏状态映射到输出序列的概率分布
        self.dropout = nn.Dropout(p=dropout_p)

    def forward(self,encoder_outputs,encoder_hidden,target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = th.empty(batch_size,1,dtype=th.long).fill_(SOS_token) #创建一个初始的解码器输入，填充为起始标记 开始词元，表示开始生成一个序列
        decoder_hidden = encoder_hidden #解码器的初始隐藏状态为编码器的最终隐藏状态
        decoder_outputs = [] #用于存储解码器的输出序列
        attentions = [] #用于存储注意力权重
        for i in range(MAX_LENGTH):
            decoder_output,decoder_hidden,attn_weights = self.forward_step(decoder_input,decoder_hidden,encoder_outputs) #对解码器进行一步前向传播
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights) #将注意力权重添加到注意力权重序列中
            if target_tensor is not None:
                decoder_input = target_tensor[:,i].unsqueeze(1)
            else:
                _,topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()
        decoder_outputs = th.cat(decoder_outputs,dim=1) #将解码器的输出序列拼接起来
        decoder_outputs = F.log_softmax(decoder_outputs,dim=-1) #对解吗器的输出应用softmax函数，得到概率分布
        attentions = th.cat(attentions,dim=1) #将注意力权重序列拼接起来
        return decoder_outputs,decoder_hidden,attentions #返回解码器的输出序列、最终隐藏状态和注意力权重

    def forward_step(self,input,hidden,encoder_outputs):
        embedded = self.embedding(input) #将解码器输入嵌入为密集的向量表示
        embedded = self.dropout(embedded) #在嵌入向量上应用dropout进行随机失活
        query = hidden.permute(1,0,2) #将解码器的隐藏状态进行转置，用于与编码器的输出进行注意力计算
        context,attn_weights = self.attention(query,encoder_outputs) #通过注意力机制计算上下文向量和注意力权重
        rnn_input = th.cat((embedded,context),dim=2) #将嵌入向量和上下文向量进行拼接，作为RNN的输入
        output,hidden = self.rnn(rnn_input,hidden) #通过RNN层处理拼接后的输入，得到输出和最终隐藏状态
        output = self.output(output) #通过全连接层将RNN的输出映射到输出序列的概率分布
        return output,hidden,attn_weights #返回解码器的输出、最终隐藏状态和注意力权重


In [21]:
decoder = AttentionDecoderRNN(hidden_size=5,output_size=10)
target_seq = th.tensor([[0,1,2,3,4,5,6,7,8,9]])
encoder_outputs, encoder_hidden = encoder(input_seq)
output, hidden,attn_weights = decoder(encoder_outputs,encoder_hidden,input_seq) #通过解码器进行前向传播
print('输出向量的维度：',output.size())
print('注意力权重的维度：',attn_weights.size())

输出向量的维度： torch.Size([1, 10, 10])
注意力权重的维度： torch.Size([1, 10, 10])


In [12]:
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import datetime

#实战：日期转换 将中文的‘年-月-日’格式的日期转换为英文格式‘day/month/year’ 区间是1950-2050年 增加难度，中文提供‘YY-MM-DD’ 缺少前面2位，模型需要推测出前面的2位数字
class DateDataset(Dataset):
    def __init__(self, n):
        # 初始化两个空列表，用于存储中文和英文日期
        self.date_cn = []
        self.date_en = []
        for _ in range(n):
            # 随机生成年份、月份和日期
            year = random.randint(1950, 2050)
            month = random.randint(1, 12)
            day = random.randint(1, 28)  # 假设最大日期为28日
            date = datetime.date(year, month, day)
            # 格式化日期并添加到对应的列表中
            self.date_cn.append(date.strftime("%y-%m-%d"))
            self.date_en.append(date.strftime("%d/%b/%Y"))
        # 创建一个词汇集，包含0-9的数字、"-"、"/"和英文日期中的月份缩写
        self.vocab = set([str(i) for i in range(0, 10)] +
                         ["-", "/"] + [i.split("/")[1] for i in self.date_en])
        # 创建一个词汇到索引的映射，其中"<SOS>"、"<EOS>"和"<PAD>"分别对应开始、结束和填充标记
        self.word2index = {v: i for i, v in enumerate(sorted(list(self.vocab)), start=3)}
        self.word2index["<PAD>"] = PAD_token
        self.word2index["<SOS>"] = SOS_token
        self.word2index["<EOS>"] = EOS_token
        # 将开始、结束和填充标记添加到词汇集中
        self.vocab.add("<SOS>")
        self.vocab.add("<EOS>")
        self.vocab.add("<PAD>")
        # 创建一个索引到词汇的映射
        self.index2word = {i: v for v, i in self.word2index.items()}
        # 初始化输入和目标列表
        self.input, self.target = [], []
        for cn, en in zip(self.date_cn, self.date_en):
            # 将日期字符串转换为词汇索引列表，然后添加到输入和目标列表中
            self.input.append([self.word2index[v] for v in cn])
            self.target.append(
                [self.word2index["<SOS>"], ] +
                [self.word2index[v] for v in en[:3]] +
                [self.word2index[en[3:6]]] +
                [self.word2index[v] for v in en[6:]] +
                [self.word2index["<EOS>"], ]
            )
        # 将输入和目标列表转换为NumPy数组
        self.input, self.target = np.array(self.input), np.array(self.target)

    def __len__(self):
        # 返回数据集的长度，即输入的数量
        return len(self.input)

    def __getitem__(self, index):
        # 返回给定索引的输入、目标和目标的长度
        return self.input[index], self.target[index], len(self.target[index])

    @property
    def num_word(self):
        # 返回词汇表的大小
        return len(self.vocab)

In [17]:
dataset = DateDataset(1000)

In [18]:
dataset.date_cn[:5], dataset.date_en[:5]

(['29-07-07', '41-06-27', '66-12-01', '37-08-17', '29-10-25'],
 ['07/Jul/2029', '27/Jun/2041', '01/Dec/1966', '17/Aug/2037', '25/Oct/2029'])

In [19]:
n_epochs = 100
batch_size = 32
MAX_LENGTH = 11
hidden_size = 128
learning_rate = 0.001

In [15]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True,drop_last=True)
encoder = EncoderRNN(input_size=dataset.num_word, hidden_size=hidden_size)
decoder = AttentionDecoderRNN(hidden_size=hidden_size, output_size=dataset.num_word)
encoder_optimizer = th.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = th.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()

NameError: name 'dataset' is not defined

In [38]:
for i in range(n_epochs+1):
    total_loss = 0
    for input_seq, target_seq,target_length in dataloader:
        encoder_optimizer.zero_grad() #梯度清零
        decoder_optimizer.zero_grad()
        encoder_outputs, encoder_hidden = encoder(input_seq) #通过编码器进行前向传播
        decoder_outputs, _, _ = decoder(encoder_outputs,encoder_hidden,target_seq) #通过解码器进行前向传播
        loss = criterion(decoder_outputs.view(-1, decoder_outputs.size(-1)), target_seq.view(-1).long()) #计算损失
        loss.backward()
        encoder_optimizer.step() #更新编码器参数
        decoder_optimizer.step() #更新解码器参数
        total_loss += loss.item() #累加损失

    total_loss = total_loss/len(dataloader)
    if i % 10 == 0:
        print(f'Epoch [{i}/{n_epochs}], Loss: {total_loss:.4f}')

Epoch [0/100], Loss: 1.9712
Epoch [10/100], Loss: 0.0116
Epoch [20/100], Loss: 0.0035
Epoch [30/100], Loss: 0.0022
Epoch [40/100], Loss: 0.0010
Epoch [50/100], Loss: 0.0005
Epoch [60/100], Loss: 0.0004
Epoch [70/100], Loss: 0.0003
Epoch [80/100], Loss: 0.0002
Epoch [90/100], Loss: 0.0002
Epoch [100/100], Loss: 0.0025


In [39]:
#评估模式
def evaluate(encoder, decoder,x):
    encoder.eval()
    decoder.eval()
    encoder_outputs, encoder_hidden = encoder(th.tensor(np.array([x])))
    start = th.ones(x.shape[0],1) #创建一个初始的解码器输入，填充为起始标记 开始词元，表示开始生成一个序列
    start[:,0] = th.tensor([SOS_token]).long()
    decoder_outputs, _, _ = decoder(encoder_outputs,encoder_hidden)
    _, topi = decoder_outputs.topk(1)
    decoded_ids = topi.squeeze()
    decoded_words =[]
    for idx in decoded_ids:
        decoded_words.append(dataset.index2word[idx.item()])
    return "".join(decoded_words)

for i in range(5):
    predict = evaluate(encoder, decoder, dataset[i][0])
    print(f"input:{dataset.date_cn[i]}, target:{dataset.date_en[i]}, prediction:{predict}")


input:14-09-16, target:16/Sep/2014, prediction:<SOS>16/Sep/2014<EOS>
input:65-05-09, target:09/May/1965, prediction:<SOS>09/May/1965<EOS>
input:58-09-10, target:10/Sep/1958, prediction:<SOS>10/Sep/1958<EOS>
input:91-11-26, target:26/Nov/1991, prediction:<SOS>26/Nov/1991<EOS>
input:59-11-04, target:04/Nov/1959, prediction:<SOS>04/Nov/1959<EOS>


### 2. Transformer架构

In [2]:
from torch.nn.functional import cross_entropy, softmax, relu
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import torch as th
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, model_dim, drop_rate):
        super().__init__()
        # 每个注意力头的维度
        self.head_dim = model_dim // n_head
        # 注意力头的数量
        self.n_head = n_head
        # 模型的维度
        self.model_dim = model_dim
        # 初始化线性变换层，用于生成query、key和value
        self.wq = nn.Linear(model_dim, n_head * self.head_dim)
        self.wk = nn.Linear(model_dim, n_head * self.head_dim)
        self.wv = nn.Linear(model_dim, n_head * self.head_dim)
        # 输出的全连接层
        self.output_dense = nn.Linear(model_dim, model_dim)
        # Dropout层，用于防止模型过拟合
        self.output_drop = nn.Dropout(drop_rate)
        # 层归一化，用于稳定神经网络的训练
        self.layer_norm = nn.LayerNorm(model_dim)
        self.attention = None

    def forward(self, q, k, v, mask):
        # 保存原始输入q，用于后续的残差连接
        residual = q
        # 分别对输入q,k,v做线性变换，生成query、key和value
        query = self.wq(q)
        key   = self.wk(k)
        value = self.wv(v)
        # 对生成的query、key和value进行头分割，以便进行多头注意力计算
        query = self.split_heads(query)
        key   = self.split_heads(key)
        value = self.split_heads(value)
        # 计算上下文向量
        context = self.scaled_dot_product_attention(query, key, value, mask)
        # 对上下文向量进行线性变换
        output = self.output_dense(context)
        # 添加dropout
        output = self.output_drop(output)
        # 添加残差连接并进行层归一化
        output = self.layer_norm(residual + output)
        return output

    def split_heads(self, x):
        # 将输入x的shape变为(n, step, n_head, head_dim)，然后进行重排，得到(n, n_head, step, head_dim)
        x = th.reshape(x, (x.shape[0], x.shape[1], self.n_head, self.head_dim))
        return x.permute(0, 2, 1, 3)

    #缩放点积注意力是一种注意力机制，用于计算序列中不同位置之间的相关性。缩放的目的是为了避免在计算注意力分数时出现数值不稳定的情况。从而导致在后续训练中的梯度消失或梯度爆炸的问题。
    def scaled_dot_product_attention(self, q, k, v, mask=None):
        # 计算缩放因子
        dk = th.tensor(k.shape[-1]).type(th.float)
        # 计算注意力分数
        score = th.matmul(q, k.permute(0, 1, 3, 2)) / (th.sqrt(dk) + 1e-8)
        if mask is not None:
            # 如果提供了mask，将mask位置的分数设置为负无穷，使得这些位置的softmax值接近0
            score = score.masked_fill_(mask,-np.inf)
        # 计算softmax得到注意力权重
        self.attention = softmax(score,dim=-1)
        # 计算上下文向量
        context = th.matmul(self.attention,v)
        # 重排上下文向量的维度并进行维度合并
        context = context.permute(0, 2, 1, 3)
        context = context.reshape((context.shape[0], context.shape[1],-1))
        return context

In [4]:
class PositionWiseFFN(nn.Module):
    def __init__(self, model_dim, dropout=0.0):
        super().__init__()
        # 前馈神经网络的隐藏层维度，设为模型维度的4倍
        ffn_dim = model_dim * 4
        # 第一个线性变换层，其输出维度为前馈神经网络的隐藏层维度
        self.linear1 = nn.Linear(model_dim, ffn_dim)
        # 第二个线性变换层，其输出维度为模型的维度
        self.linear2 = nn.Linear(ffn_dim, model_dim)
        # Dropout层，用于防止模型过拟合
        self.dropout = nn.Dropout(dropout)
        # 层归一化，用于稳定神经网络的训练
        self.layer_norm = nn.LayerNorm(model_dim)

    def forward(self, x):
        # 对输入x进行前馈神经网络的计算
        # 首先，通过第一个线性变换层并使用relu作为激活函数
        output = relu(self.linear1(x))
        # 然后，通过第二个线性变换层
        output = self.linear2(output)
        # 接着，对输出做dropout
        output = self.dropout(output)
        # 最后，对输入x和前馈神经网络的输出做残差连接，然后进行层归一化
        output = self.layer_norm(x + output)
        return output  # 返回结果，其shape为[n, step, dim]

In [5]:
class EncoderLayer(nn.Module):
    def __init__(self, n_head, emb_dim, drop_rate):
        super().__init__()
        # 多头注意力机制层
        self.mha = MultiHeadAttention(n_head, emb_dim, drop_rate)
        # 前馈神经网络层
        self.ffn = PositionWiseFFN(emb_dim, drop_rate)

    def forward(self, xz, mask):
        # xz的形状为 [n, step, emb_dim]
        # 通过多头注意力机制层处理xz，得到context，其形状也为 [n, step, emb_dim]
        context = self.mha(xz, xz, xz, mask)
        # 将context传入前馈神经网络层，得到输出
        output = self.ffn(context)
        return output


class Encoder(nn.Module):
    def __init__(self, n_head, emb_dim, drop_rate, n_layer):
        super().__init__()
        # 定义n_layer个EncoderLayer，保存在一个ModuleList中
        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(n_head, emb_dim, drop_rate) for _ in range(n_layer)]
        )

    def forward(self, xz, mask):
        # 依次通过所有的EncoderLayer
        for encoder in self.encoder_layers:
            xz = encoder(xz, mask)
        return xz  # 返回的xz形状为 [n, step, emb_dim]

In [6]:
class DecoderLayer(nn.Module):
    def __init__(self, n_head, model_dim, drop_rate):
        super().__init__()
        # 定义两个多头注意力机制层
        self.mha = nn.ModuleList([MultiHeadAttention(n_head, model_dim, drop_rate) for _ in range(2)])
        # 定义一个前馈神经网络层
        self.ffn = PositionWiseFFN(model_dim, drop_rate)

    def forward(self, yz, xz, yz_look_ahead_mask, xz_pad_mask):
        # 第一个注意力层的计算，三个输入均为yz，使用自注意力机制
        dec_output = self.mha[0](yz, yz, yz, yz_look_ahead_mask)  # [n, step, model_dim]
        # 第二个注意力层的计算，其中q来自前一个注意力层的输出，K和V来自编码器的输出
        dec_output = self.mha[1](dec_output, xz, xz, xz_pad_mask)  # [n, step, model_dim]
        # 通过前馈神经网络层
        dec_output = self.ffn(dec_output)   # [n, step, model_dim]
        return dec_output


class Decoder(nn.Module):
    def __init__(self, n_head, model_dim, drop_rate, n_layer):
        super().__init__()
        # 定义n_layer个DecoderLayer，保存在一个ModuleList中
        self.num_layers = n_layer
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(n_head, model_dim, drop_rate) for _ in range(n_layer)]
        )

    def forward(self, yz, xz, yz_look_ahead_mask, xz_pad_mask):
        # 依次通过所有的DecoderLayer
        for decoder in self.decoder_layers:
            yz = decoder(yz, xz, yz_look_ahead_mask, xz_pad_mask)
        return yz  # 返回的yz形状为 [n, step, model_dim]

In [7]:
class PositionEmbedding(nn.Module):
    def __init__(self, max_len, emb_dim, n_vocab):
        super().__init__()
        # 生成位置编码矩阵
        pos = np.expand_dims(np.arange(max_len), 1)  # [max_len, 1]
        # 使用正弦和余弦函数生成位置编码
        pe = pos / np.power(1000, 2*np.expand_dims(np.arange(emb_dim)//2, 0)/emb_dim)
        pe[:, 0::2] = np.sin(pe[:, 0::2])
        pe[:, 1::2] = np.cos(pe[:, 1::2])
        pe = np.expand_dims(pe, 0)  # [1, max_len, emb_dim]
        self.pe = th.from_numpy(pe).type(th.float32)

        # 定义词嵌入层
        self.embeddings = nn.Embedding(n_vocab, emb_dim)
        # 初始化词嵌入层的权重
        self.embeddings.weight.data.normal_(0, 0.1)

    def forward(self, x):
        # 确保位置编码在与词嵌入权重相同的设备上
        device = self.embeddings.weight.device
        self.pe = self.pe.to(device)
        # 计算输入的词嵌入，并加上位置编码
        x_embed = self.embeddings(x) + self.pe  # [n, step, emb_dim]
        return x_embed  # [n, step, emb_dim]

In [8]:
def pad_zero(seqs, max_len):
    # 初始化一个全是填充标识符PAD_token的二维矩阵，大小为(len(seqs), max_len)
    padded = np.full((len(seqs), max_len), fill_value=PAD_token, dtype=np.int32)
    for i, seq in enumerate(seqs):
        # 将seqs中的每个序列seq的元素填入padded对应的行中，未填满的部分仍为PAD_token
        padded[i, :len(seq)] = seq
    return padded

In [9]:
class Transformer(nn.Module):
    def __init__(self, n_vocab, max_len, n_layer=6, emb_dim=512, n_head=8, drop_rate=0.1, padding_idx=0):
        super().__init__()
        # 初始化最大长度、填充索引、词汇表大小
        self.max_len = max_len
        self.padding_idx = th.tensor(padding_idx)
        self.dec_v_emb = n_vocab
        # 初始化位置嵌入、编码器、解码器和输出层
        self.embed = PositionEmbedding(max_len, emb_dim, n_vocab)
        self.encoder = Encoder(n_head, emb_dim, drop_rate, n_layer)
        self.decoder = Decoder(n_head, emb_dim, drop_rate, n_layer)
        self.output = nn.Linear(emb_dim, n_vocab)
        # 初始化优化器
        self.opt = th.optim.Adam(self.parameters(), lr=0.002)

    def forward(self, x, y):
        # 对输入和目标进行嵌入
        x_embed, y_embed = self.embed(x), self.embed(y)
        # 创建填充掩码
        pad_mask = self._pad_mask(x)
        # 对输入进行编码
        encoded_z = self.encoder(x_embed, pad_mask)
        # 创建前瞻掩码
        yz_look_ahead_mask = self._look_ahead_mask(y)
        # 将编码后的输入和前瞻掩码传入解码器
        decoded_z = self.decoder(
            y_embed, encoded_z, yz_look_ahead_mask, pad_mask)
        # 通过输出层得到最终输出
        output = self.output(decoded_z)
        return output

    def step(self, x, y):
        # 清空梯度
        self.opt.zero_grad()
        # 计算输出和损失
        logits = self(x, y[:, :-1])
        loss = cross_entropy(logits.reshape(-1, self.dec_v_emb), y[:, 1:].reshape(-1))
        # 进行反向传播
        loss.backward()
        # 更新参数
        self.opt.step()
        return loss.cpu().data.numpy(), logits

    def _pad_bool(self, seqs):
        # 创建掩码，标记哪些位置是填充的
        return th.eq(seqs, self.padding_idx)

    def _pad_mask(self, seqs):
        # 将填充掩码扩展到合适的维度
        len_q = seqs.size(1)
        mask = self._pad_bool(seqs).unsqueeze(1).expand(-1, len_q, -1)
        return mask.unsqueeze(1)

    def _look_ahead_mask(self, seqs):
        # 创建前瞻掩码，防止在生成序列时看到未来的信息
        device = next(self.parameters()).device
        _, seq_len = seqs.shape
        mask = th.triu(th.ones((seq_len, seq_len), dtype=th.long),
                       diagonal=1).to(device)
        mask = th.where(self._pad_bool(seqs)[:, None, None, :], 1, mask[None, None, :, :]).to(device)
        return mask > 0

In [20]:
# 初始化一个Transformer模型，设置词汇表大小、最大序列长度、层数、嵌入维度、多头注意力的头数、 dropout比率和填充标记的索引
model = Transformer(n_vocab=dataset.num_word, max_len=MAX_LENGTH, n_layer=3, emb_dim=32, n_head=8, drop_rate=0.1, padding_idx=0)
# 检测是否有可用的GPU，如果有，则使用GPU进行计算；如果没有，则使用CPU
device = th.device("cuda" if th.cuda.is_available() else "cpu")
# 将模型移动到相应的设备（CPU或GPU）
model = model.to(device)
# 创建一个数据集，包含1000个样本
dataset = DateDataset(1000)
# 创建一个数据加载器，设定批次大小为32，每个批次的数据会被打乱
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
# 进行10个训练周期
for i in range(10):
    # 对于数据加载器中的每批数据
    for input_tensor, target_tensor, _ in dataloader:
        # 对输入和目标张量进行零填充，使其长度达到最大长度，然后将其转换为PyTorch张量，并移动到相应的设备（CPU或GPU）
        input_tensor = th.from_numpy(
            pad_zero(input_tensor, max_len=MAX_LENGTH)).long().to(device)
        target_tensor = th.from_numpy(
            pad_zero(target_tensor, MAX_LENGTH+1)).long().to(device)
        # 使用模型的step方法进行一步训练，并获取损失值
        loss, _ = model.step(input_tensor, target_tensor)
    # 打印每个训练周期后的损失值
    print(f"epoch: {i+1}, \tloss: {loss}")

epoch: 1, 	loss: 1.4655067920684814
epoch: 2, 	loss: 1.0785928964614868
epoch: 3, 	loss: 0.9121214151382446
epoch: 4, 	loss: 0.6275089383125305
epoch: 5, 	loss: 0.34216246008872986
epoch: 6, 	loss: 0.24774736166000366
epoch: 7, 	loss: 0.15374785661697388
epoch: 8, 	loss: 0.06169423833489418
epoch: 9, 	loss: 0.03446295112371445
epoch: 10, 	loss: 0.017239874228835106


In [21]:
def evaluate(model, x, y):
    model.eval()
    x = th.from_numpy(pad_zero([x], max_len=MAX_LENGTH)).long().to(device)
    y = th.from_numpy(pad_zero([y], max_len=MAX_LENGTH)).long().to(device)
    decoder_outputs = model(x, y)
    _, topi = decoder_outputs.topk(1)
    decoded_ids = topi.squeeze()
    decoded_words = []
    for idx in decoded_ids:
        decoded_words.append(dataset.index2word[idx.item()])
    return ''.join(decoded_words)

In [22]:
for i in range(5):
    predict = evaluate(model, dataset[i][0], dataset[i][1])
    print(f"input: {dataset.date_cn[i]}, target: {dataset.date_en[i]}, predict: {predict}")

input: 56-09-06, target: 06/Sep/1956, predict: 06/Sep/1956<EOS><PAD>
input: 09-07-22, target: 22/Jul/2009, predict: 22/Jul/2009<EOS><PAD>
input: 66-01-23, target: 23/Jan/1966, predict: 23/Jan/1966<EOS><PAD>
input: 92-07-22, target: 22/Jul/1992, predict: 22/Jul/1992<EOS><PAD>
input: 58-07-12, target: 12/Jul/1958, predict: 12/Jul/1958<EOS><PAD>
